Let's try some basic tests of `VB.jl`:

In [1]:
include("VB.jl")
using VB
using Distributions

# Make a node

In [2]:
μ = rand(5, 5)
σ = rand(5, 5)
aa = RandomNode(:a, [:i, :j], Normal, μ, σ)
bb = RandomNode(:b, [:j, :k], Gamma, rand(5, 3), rand(5, 3))

VB.RandomNode{Distributions.Gamma}(:b,Symbol[],[:j,:k],5x3 Array{Distributions.Gamma,2}:
 Distributions.Gamma(α=0.5816654249399293, θ=0.8118559608309102)    …  Distributions.Gamma(α=0.3659952907787327, θ=0.10180438531923763)
 Distributions.Gamma(α=0.41945014632224975, θ=0.5854950570843731)      Distributions.Gamma(α=0.872218077405007, θ=0.5292864535009745)  
 Distributions.Gamma(α=0.46015825907991803, θ=0.2643074710486386)      Distributions.Gamma(α=0.8944299821690787, θ=0.32016065056476095)
 Distributions.Gamma(α=0.23558590039223604, θ=0.8605725300282003)      Distributions.Gamma(α=0.7911209969577011, θ=0.6166906448938994) 
 Distributions.Gamma(α=0.7049408919115381, θ=0.012859864310270641)     Distributions.Gamma(α=0.3511608201975418, θ=0.8225775598029583) )

In [3]:
cc = ConstantNode(:c, [:i, :j], rand(3, 4))

VB.ConstantNode{Float64}(:c,Symbol[],[:i,:j],3x4 Array{Float64,2}:
 0.918864  0.227525   0.906902  0.330157
 0.02747   0.0338514  0.267962  0.609874
 0.891026  0.80223    0.76398   0.249833)

In [4]:
cc = ConstantNode(rand(3, 4), [:i, :j])

VB.ConstantNode{Float64}(symbol("##const#7769"),Symbol[],[:i,:j],3x4 Array{Float64,2}:
 0.43758    0.203854  0.661685  0.570825
 0.662014   0.300279  0.673258  0.925846
 0.0131563  0.84656   0.37589   0.352118)

In [5]:
nodes = Node[aa, bb]
fi = get_structure(nodes...)
fi

VB.FactorInds([:i,:j,:k],[5,5,3],Dict(:a=>[1,2],:b=>[2,3]),Dict(:a=>[1,2],:b=>[1,2]))

In [6]:
a[i, j] ~ Normal(μ, σ)

VB.RandomNode{Distributions.Normal}(:a,Symbol[],[:i,:j],5x5 Array{Distributions.Normal,2}:
 Distributions.Normal(μ=0.9595176958009373, σ=0.9413540573472496)   …  Distributions.Normal(μ=0.5158919023258492, σ=0.07401571465540324)
 Distributions.Normal(μ=0.08495008725807707, σ=0.8775271163579395)     Distributions.Normal(μ=0.3417226685462347, σ=0.8573647663277206) 
 Distributions.Normal(μ=0.9667091205619043, σ=0.9952332191495259)      Distributions.Normal(μ=0.5673580491985311, σ=0.8677224720381846) 
 Distributions.Normal(μ=0.8782725823807305, σ=0.3707667370460299)      Distributions.Normal(μ=0.6918673208378425, σ=0.13326297251622687)
 Distributions.Normal(μ=0.9073901486262097, σ=0.8820837139485764)      Distributions.Normal(μ=0.31478632316797284, σ=0.803139124662344) )

In [7]:
c[i, j] ~ Const(rand(3, 4))

VB.ConstantNode{Float64}(:c,Symbol[],[:i,:j],3x4 Array{Float64,2}:
 0.0381583  0.414331   0.880829  0.819818 
 0.571025   0.0634958  0.811855  0.469022 
 0.55277    0.291532   0.88283   0.0877853)

Make a multivariate node

In [8]:
dims = (5, 3)
m = [rand(dims[1]) for x in 1:dims[2]]
VV = [diagm(rand(dims[1])) for x in 1:dims[2]]
d[i, j] ~ MvNormal(m, VV)
d.innerinds, d.outerinds

([:i],[:j])

# Make a factor

In [9]:
dims = (10, 2)

μ[j] ~ Normal(rand(dims[2]), ones(dims[2]))
τ ~ Gamma(1, 1)
x[i, j] ~ Normal(rand(dims), ones(dims))

f = @factor LogNormalFactor x μ τ;
value(f)

-34.19953709445545

and a multivariate factor...

In [10]:
dims = (5, 3)
m = [rand(dims[1]) for x in 1:dims[2]]
VV = [diagm(rand(dims[1])) for x in 1:dims[2]]

y[i, j] ~ MvNormal(m, VV)
μ[i] ~ Normal(zeros(dims[1]), ones(dims[1]))
Σ[j] ~ Const([diagm(rand(dims[1])) for x in 1:dims[2]])

f = @factor LogMvNormalCanonFactor y μ Σ
value(f)

-27.67887571887946

In [11]:
vars = fieldnames(f)
val_expr = value(typeof(f))
using Base.Cartesian
macroexpand(:(@wrapvars $vars $val_expr (@ntuple 1 i)))

quote  # /Users/jmxp/code/vbgraph/types.jl, line 380:
    δ = E(project(f,:x,(i_1,))) - E(project(f,:μ,(i_1,))) # /Users/jmxp/code/vbgraph/types.jl, line 381:
    EΛ = E(project(f,:Λ,(i_1,))) # /Users/jmxp/code/vbgraph/types.jl, line 382:
    -(1 / 2) * ((trace(EΛ * ((V(project(f,:x,(i_1,))) .+ V(project(f,:μ,(i_1,)))) .+ δ * δ')) + length(project(f,:x,(i_1,))) * log(2π)) - Elogdet(project(f,:Λ,(i_1,))))
end

In [12]:
f.inds.indices, f.inds.inds_in_factor, f.inds.inds_in_node

([:j],Dict(:μ=>Int64[],:y=>[1],:Σ=>[1]),Dict(:μ=>Int64[],:y=>[1],:Σ=>[1]))

In [13]:
project_inds(f, :x, (5,)), project_inds(f, :μ, (5,))

(Any[5],Any[Colon()])

In [14]:
μ

VB.RandomNode{Distributions.Normal}(:μ,Symbol[],[:i],[Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0)])

In [15]:
g = @factor EntropyFactor x
value(g)

28.378770664093462

# Let's make a simple model

We want a simple model of inference for a normal distribution:

$$
\begin{aligned}
    y \sim \mathcal{N}(\mu, \tau) \\
    \mu \sim \mathcal{N}(\mu_0, \tau_0) \\
    \tau \sim \mathrm{Ga}(\alpha, \beta) \\
    q(\mu) = \mathcal{N}(m, t) \\
    q(\tau) = \mathrm{Ga}(a, b)
\end{aligned}
$$

This model is conjugate, and can be solved by straightforward updates of the natural parameters of the posterior.

## Make the nodes

We need to make nodes for all the random variables that will need to be updated (we can also create nodes for parameter arrays, but these will be converted automatically if we don't).

In [16]:
dims = (20, 6)

# note: it won't matter much how we initialize here
μ[j] ~ Normal(zeros(dims[2]), ones(dims[2]))
τ[j] ~ Gamma(1.1 * ones(dims[2]), ones(dims[2]))
μ0[j] ~ Const(zeros(dims[2]))
τ0[j] ~ Const(2 * ones(dims[2]))
a0[j] ~ Const(1.1 * ones(dims[2]))
b0[j] ~ Const(ones(dims[2]))

y[i, j] ~ Const(rand(dims));

Now make factors: We need a Normal factor for the observation model plus a prior and an entropy for each node.

In [17]:
obs = @factor LogNormalFactor y μ τ
μ_prior = @factor LogNormalFactor μ μ0 τ0
τ_prior = @factor LogGammaFactor τ a0 b0

VB.LogGammaFactor{1}(VB.RandomNode{Distributions.Gamma}(:τ,Symbol[],[:j],[Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0)]),VB.ConstantNode{Float64}(:a0,Symbol[],[:j],[1.1,1.1,1.1,1.1,1.1,1.1]),VB.ConstantNode{Float64}(:b0,Symbol[],[:j],[1.0,1.0,1.0,1.0,1.0,1.0]),VB.FactorInds([:j],[6],Dict(:τ=>[1],:b0=>[1],:a0=>[1]),Dict(:τ=>[1],:b0=>[1],:a0=>[1])),Dict(:τ=>:x,:b0=>:β,:a0=>:α))

In [18]:
value(obs), value(μ_prior), value(τ_prior)

(-174.1830520792865,-13.593072740907873,0.0449816833123928)

In [19]:
m = VBModel([μ, τ, μ0, τ0, a0, b0, y], [obs, μ_prior, τ_prior])

VB.VBModel(VB.Node[VB.RandomNode{Distributions.Normal}(:μ,Symbol[],[:j],[Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0)]),VB.RandomNode{Distributions.Gamma}(:τ,Symbol[],[:j],[Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0)]),VB.ConstantNode{Float64}(:μ0,Symbol[],[:j],[0.0,0.0,0.0,0.0,0.0,0.0]),VB.ConstantNode{Float64}(:τ0,Symbol[],[:j],[2.0,2.0,2.0,2.0,2.0,2.0]),VB.ConstantNode{Float64}(:a0,Symbol[],[:j],[1.1,1.1,1.1,1.1,1.1,1.1]),VB.ConstantNode{Float64}(:b0,Symbol[],[:j],[1.0,1.0,1.0,1.0,1.0,1.0]),VB.ConstantNode{Float64}(:y,Symbol[],[:i,:j],20x6 Array{Float64,2}:
 0.0733417  0.593968    0.54841   0.323937    0.0588273  0.45829  
 0.307873   0.930857    0.212867  0.27210

In [20]:
[n.name for n in m.nodes]

7-element Array{Any,1}:
 :μ 
 :τ 
 :μ0
 :τ0
 :a0
 :b0
 :y 

In [21]:
[typeof(f) for f in m.factors]

3-element Array{DataType,1}:
 VB.LogNormalFactor{2}
 VB.LogNormalFactor{1}
 VB.LogGammaFactor{1} 

In [22]:
m.graph

Dict{VB.Node,Array{Tuple{VB.Factor{N},Symbol},1}} with 7 entries:
  VB.RandomNode{Distribut… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{2}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{1}…
  VB.RandomNode{Distribut… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{2}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{2}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{1}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogGammaFactor{1}(…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogGammaFactor{1}(…

In [23]:
naturals(obs, μ)

6-element Array{Tuple{Float64,Float64},1}:
 (10.039151872457056,-11.000000000000002)
 (10.814368111401961,-11.000000000000002)
 (12.67040485711798,-11.000000000000002) 
 (11.837253077636348,-11.000000000000002)
 (12.039452279806952,-11.000000000000002)
 (11.365427183892832,-11.000000000000002)

In [24]:
naturals(obs, τ)

6-element Array{Tuple{Float64,Float64},1}:
 (10.0,13.045277834716133)
 (10.0,13.469760036780878)
 (10.0,13.96990553016444) 
 (10.0,13.710621354707342)
 (10.0,13.56386184870914) 
 (10.0,13.454868412549605)

In [25]:
check_conjugate(τ, m)

true

In [26]:
check_conjugate(μ, m)

true

In [27]:
μ.data

6-element Array{Distributions.Normal,1}:
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)

In [28]:
τ.data

6-element Array{Distributions.Gamma,1}:
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)

In [29]:
update!(m)

In [30]:
μ.data

6-element Array{Distributions.Normal,1}:
 Distributions.Normal(μ=-0.6794222266760135, σ=0.7071067811865476)
 Distributions.Normal(μ=-0.1312615672321401, σ=0.7071067811865476)
 Distributions.Normal(μ=1.1811546017950683, σ=0.7071067811865476) 
 Distributions.Normal(μ=0.5920273287659671, σ=0.7071067811865476) 
 Distributions.Normal(μ=0.7350037557713109, σ=0.7071067811865476) 
 Distributions.Normal(μ=0.2583960397605238, σ=0.7071067811865476) 

In [31]:
τ.data

6-element Array{Distributions.Gamma,1}:
 Distributions.Gamma(α=29.862171564024095, θ=0.8999999999999999)
 Distributions.Gamma(α=20.932520487946803, θ=0.8999999999999999)
 Distributions.Gamma(α=20.31597927849793, θ=0.8999999999999999) 
 Distributions.Gamma(α=16.84469646251067, θ=0.8999999999999999) 
 Distributions.Gamma(α=16.92158283769976, θ=0.8999999999999999) 
 Distributions.Gamma(α=17.45275229663927, θ=0.8999999999999999) 

# A Multivariate Model

We want a simple model of inference for a multivariate normal distribution:

$$
\begin{aligned}
    y \sim \mathcal{N}(\pmb{\mu}, \Sigma) \\
    \mu_i \sim \mathcal{N}(\mu_0, \tau_0) \\
    \tau \sim \mathrm{Ga}(\alpha, \beta) \\
    q(\mu) = \mathcal{N}(m, t) \\
    q(\tau) = \mathrm{Ga}(a, b)
\end{aligned}
$$

This model is conjugate, and can be solved by straightforward updates of the natural parameters of the posterior.

In [32]:
d = 5
μ[i] ~ MvNormalCanon(zeros(d), diagm(ones(d)))
Λ[i, i] ~ Wishart(float(d), diagm(ones(d)))
x[i, j] ~ Const(randn(d, 20))
f = @factor LogMvNormalCanonFactor x μ Λ

VB.LogMvNormalCanonFactor{2}(VB.ConstantNode{Float64}(:x,Symbol[],[:i,:j],5x20 Array{Float64,2}:
 -1.82061    0.755927   -0.329262  …  -0.565857  -2.16483     0.611639
  0.495277  -1.12082    -0.935893      0.911396   0.945788   -0.775987
  0.875223   1.30076     0.335687      3.30343    0.0965557   0.436796
  0.270513  -0.227459    0.673672     -0.682049   0.878979   -0.565615
 -0.78557    0.0686703  -0.663341     -1.5172     0.084306    0.888977),VB.RandomNode{Distributions.MvNormalCanon{P<:PDMats.AbstractPDMat,V<:Union{Array{Float64,1},Distributions.ZeroVector{Float64}}}}(:μ,[:i],[:scalar],Distributions.MvNormalCanon[FullNormalCanon(
μ: [0.0,0.0,0.0,0.0,0.0]
h: [0.0,0.0,0.0,0.0,0.0]
J: PDMats.PDMat(5,5x5 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  1.0,5x5 UpperTriangular{Float64,Array{Float64,2}}:
 1.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.

In [33]:
naturals(f, μ)

Base.LinAlg.Cholesky{Float64,Array{Float64,2}} with factor:


1-element Array{Tuple{Array{Float64,1},Array{Float64,2}},1}:
 ([-25.272632117027484,13.251963801349682,-0.13821469649748241,12.150865909430395,-7.36725448150872],
5x5 Array{Float64,2}:
 -50.0    0.0    0.0    0.0    0.0
   0.0  -50.0    0.0    0.0    0.0
   0.0    0.0  -50.0    0.0    0.0
   0.0    0.0    0.0  -50.0    0.0
   0.0    0.0    0.0    0.0  -50.0)

In [34]:
naturals(f, Λ)

1-element Array{Tuple{Array{Float64,2},Float64},1}:
 (
5x5 Array{Float64,2}:
 20.7311   -2.42402   -2.96336   -2.05337    3.20577 
 -2.42402  20.8515     0.804205   2.51913    0.621038
 -2.96336   0.804205  23.0868    -1.80806   -5.6423  
 -2.05337   2.51913   -1.80806   15.4567     0.931111
  3.20577   0.621038  -5.6423     0.931111  23.3147  ,

0.0)

In [35]:
d = 5
μ[i] ~ MvNormal(zeros(d), diagm(ones(d)))
        τ[i] ~ Gamma(1.1 * ones(d), ones(d))
        x[i, j] ~ Const(randn(d, 20))
        f = @factor LogMvNormalCanonFactor x μ τ


VB.LogMvNormalCanonFactor{2}(VB.ConstantNode{Float64}(:x,Symbol[],[:i,:j],5x20 Array{Float64,2}:
  0.836846   -0.636024  -0.867001   …   1.72327    0.6047     1.10175 
  0.618945    0.512347   0.302512       0.188388  -0.219246  -0.665331
 -2.12245    -0.839831   0.0751357      2.09441   -1.16972   -1.21739 
 -0.0506269  -0.606255  -0.0562736      1.05356    1.36972   -1.09185 
 -0.749102    0.911933   0.630095      -2.68685    1.14404    0.253561),VB.RandomNode{Distributions.MvNormal{Cov<:PDMats.AbstractPDMat,Mean<:Union{Array{Float64,1},Distributions.ZeroVector{Float64}}}}(:μ,[:i],[:scalar],Distributions.MvNormal[FullNormal(
dim: 5
μ: [0.0,0.0,0.0,0.0,0.0]
Σ: 5x5 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  1.0
)
]),VB.RandomNode{Distributions.Gamma}(:τ,Symbol[],[:i],[Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1

In [36]:
aa = map(params, τ.data)
bb = collect(zip(aa...))

2-element Array{Tuple{Float64,Float64,Float64,Float64,Float64},1}:
 (1.1,1.1,1.1,1.1,1.1)
 (1.0,1.0,1.0,1.0,1.0)

In [37]:
map(collect, zip(aa...))

2-element Array{Any,1}:
 [1.1,1.1,1.1,1.1,1.1]
 [1.0,1.0,1.0,1.0,1.0]

In [38]:
Array(collect(bb)[1])

LoadError: LoadError: MethodError: `convert` has no method matching convert(::Type{Array{T,N}}, ::Tuple{Float64,Float64,Float64,Float64,Float64})
This may have arisen from a call to the constructor Array{T,N}(...),
since type constructors fall back to convert methods.
Closest candidates are:
  call{T}(::Type{T}, ::Any)
  convert(::Type{Array{T,N}}, !Matched::SharedArray{T,N})
  convert{T,N}(::Type{Array{T,N}}, !Matched::AbstractArray{T,N})
  ...
while loading In[38], in expression starting on line 1

In [39]:
aa = copy(τ.data)

5-element Array{Distributions.Gamma,1}:
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)

In [40]:
dims = (20, 6)

    # note: it won't matter much how we initialize here
    μ[j] ~ Normal(zeros(dims[2]), ones(dims[2]))
    τ[j] ~ Gamma(1.1 * ones(dims[2]), ones(dims[2]))
    μ0[j] ~ Const(zeros(dims[2]))
    τ0[j] ~ Const(2 * ones(dims[2]))
    a0[j] ~ Const(1.1 * ones(dims[2]))
    b0[j] ~ Const(ones(dims[2]))

    y[i, j] ~ Const(rand(dims))

    # make factors
    obs = @factor LogNormalFactor y μ τ
    μ_prior = @factor LogNormalFactor μ μ0 τ0
    τ_prior = @factor LogGammaFactor τ a0 b0

    m = VBModel([μ, τ, μ0, τ0, a0, b0, y], [obs, μ_prior, τ_prior])

VB.VBModel(VB.Node[VB.RandomNode{Distributions.Normal}(:μ,Symbol[],[:j],[Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0)]),VB.RandomNode{Distributions.Gamma}(:τ,Symbol[],[:j],[Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0)]),VB.ConstantNode{Float64}(:μ0,Symbol[],[:j],[0.0,0.0,0.0,0.0,0.0,0.0]),VB.ConstantNode{Float64}(:τ0,Symbol[],[:j],[2.0,2.0,2.0,2.0,2.0,2.0]),VB.ConstantNode{Float64}(:a0,Symbol[],[:j],[1.1,1.1,1.1,1.1,1.1,1.1]),VB.ConstantNode{Float64}(:b0,Symbol[],[:j],[1.0,1.0,1.0,1.0,1.0,1.0]),VB.ConstantNode{Float64}(:y,Symbol[],[:i,:j],20x6 Array{Float64,2}:
 0.174979   0.476362   0.958364   0.200436  0.877307   0.927996 
 0.671342   0.962711   0.0109708  0.51822  

In [41]:
n = μ
pars_old = copy(n.data)

# use const here so that the closure below can be optimized
const fac_list = [f for (f, _) in m.graph[n]]

const x0 = unroll_pars(n)
# TODO: get lower and upper bounds from each variable in x from n

# make an objective function that sets the parameters of n to x
# and sums the values of all factors containing n
function objfun(x)
    update_pars!(n, x)
    val = 0
    for f in fac_list
        val += value(f)
    end
    val
end

gradf! = ForwardDiff.gradient(objfun, mutates=true)

# define mutating gradient; store gradient in storage array
objgrad!(x, storage) = gradf!(storage, x)


objgrad! (generic function with 1 method)

In [46]:
x2 = similar(x0)
objgrad!(x0, x2)

x2

LoadError: LoadError: InexactError()
while loading In[46], in expression starting on line 2